Allow the two machines to play for n times. One agent has weights and features and calculates the value and makes the move while the other agent makes random moves. create a dataframe with the moves made and the evaluation score. Once the playing is complete, the actual Vtrain values are populated in the dataframe. If it is the last move (game over move), the Vtrain gets 100,0,-100 (w,d,l). Otherwise, the Vtrain value is the evaluation score (estimate) of the subsequent board state. For instance, the Vtrain value of the last but one board state will be the Vestimate of the last move.

In [1]:
import numpy as np
import pandas as pd
import os
import random
from random import randint
from IPython.display import display, clear_output
import ipywidgets as widgets

In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual

## The below code is for feature extraction

In [3]:
def evalboard(test_board):
    #print("Test board inside evalboard", test_board)
    ## two squares in a row (can be row, column or diagonal) are filled with 1 and the other square is empty
    f1 = []
    
    if((np.count_nonzero(test_board[0,:])==2) & (np.sum(test_board[0,:])==2)):
        f1.append(1)
    if((np.count_nonzero(test_board[1,:])==2) & (np.sum(test_board[1,:])==2)):
        f1.append(1)
    if((np.count_nonzero(test_board[2,:])==2) & (np.sum(test_board[2,:])==2)):
        f1.append(1)
    if((np.count_nonzero(test_board[:,0])==2) & (np.sum(test_board[:,0])==2)):
        f1.append(1)
    if((np.count_nonzero(test_board[:,1])==2) & (np.sum(test_board[:,1])==2)):
        f1.append(1)
    if((np.count_nonzero(test_board[:,2])==2) & (np.sum(test_board[:,2])==2)):
        f1.append(1)
    if((np.count_nonzero(test_board.diagonal())==2) & (np.trace(test_board)==2)):
        f1.append(1)
    if((np.count_nonzero(np.fliplr(test_board).diagonal())==2) & (np.trace(np.fliplr(test_board))==2)):
        f1.append(1)

    
    
    ## two squares in a row (can be row, column or diagonal) are filled with 2 and the other square is empty
    f2 = []

    if((np.count_nonzero(test_board[0,:])==2) & (np.sum(test_board[0,:])==4)):
        f2.append(2)
    if((np.count_nonzero(test_board[1,:])==2) & (np.sum(test_board[1,:])==4)):
        f2.append(2)
    if((np.count_nonzero(test_board[2,:])==2) & (np.sum(test_board[2,:])==4)):
        f2.append(2)
    if((np.count_nonzero(test_board[:,0])==2) & (np.sum(test_board[:,0])==4)):
        f2.append(2)
    if((np.count_nonzero(test_board[:,1])==2) & (np.sum(test_board[:,1])==4)):
        f2.append(2)
    if((np.count_nonzero(test_board[:,2])==2) & (np.sum(test_board[:,2])==4)):
        f2.append(2)
    if((np.count_nonzero(test_board.diagonal())==2) & (np.trace(test_board)==4)):
        f2.append(2)
    if((np.count_nonzero(np.fliplr(test_board).diagonal())==2) & (np.trace(np.fliplr(test_board))==4)):
        f2.append(2)
    
    ## One 2 in an otherwise empty row (column, row or diagonal)
    f4 = []
    
    if((np.count_nonzero(test_board[0,:])==1) & (np.sum(test_board[0,:])==2)):
        f4.append(2)
    if((np.count_nonzero(test_board[1,:])==1) & (np.sum(test_board[1,:])==2)):
        f4.append(2)
    if((np.count_nonzero(test_board[2,:])==1) & (np.sum(test_board[2,:])==2)):
        f4.append(2)
    if((np.count_nonzero(test_board[:,0])==1) & (np.sum(test_board[:,0])==2)):
        f4.append(2)
    if((np.count_nonzero(test_board[:,1])==1) & (np.sum(test_board[:,1])==2)):
        f4.append(2)
    if((np.count_nonzero(test_board[:,2])==1) & (np.sum(test_board[:,2])==2)):
        f4.append(2)
    if((np.count_nonzero(test_board.diagonal())==1) & (np.trace(test_board)==2)):
        f4.append(2)
    if((np.count_nonzero(np.fliplr(test_board).diagonal())==1) & (np.trace(np.fliplr(test_board))==2)):
        f4.append(2)
        
    ## One 1 in an otherwise empty row (column, row or diagonal)
    f3 = []
    
    if(np.sum(test_board[0,:])==1):
        f3.append(1)
        
    if(np.sum(test_board[1,:])==1):
        f3.append(1)
        
    if(np.sum(test_board[2,:])==1):
        f3.append(1)
        
    if(np.sum(test_board[:,0])==1):
        f3.append(1)
        
    if(np.sum(test_board[:,1])==1):
        f3.append(1)
        
    if(np.sum(test_board[:,2])==1):
        f3.append(1)
        
    if(np.trace(test_board)==1):
        f3.append(1)
       
    if(np.trace(np.fliplr(test_board))==1):
        f3.append(1)
       

    

    ## Three cells in a row is filled with 1
    f5 = []
    
    if(test_board[0,:]==1).all():
        f5.append(1)
    if(test_board[1,:]==1).all():
        f5.append(1)
    if(test_board[2,:]==1).all():
        f5.append(1)
    if(test_board.diagonal()==1).all():
        f5.append(1)
    if(np.fliplr(test_board).diagonal()==1).all():
        f5.append(1)

    
    ## Three cells in a row is filled with 2
    f6 = []

    
    if(test_board[0,:]==2).all():
        f6.append(2)
    if(test_board[1,:]==2).all():
        f6.append(2)
    if(test_board[2,:]==2).all():
        f6.append(2)
    if(test_board.diagonal()==2).all():
        f6.append(2)
    if(np.fliplr(test_board).diagonal()==2).all():
        f6.append(2)
    
    return(len(f1),len(f2),len(f3),len(f4),len(f5),len(f6))

### The below function is for making the moves of the machine which is trained to become smart

In [4]:
def machine1_move(old_board_state,w0,w1,w2,w3,w4,w5,w6):
    feature_list = []
    board_state = np.copy(old_board_state)
    x, y = np.where(board_state==0)
    #print("weights are ",w0,w1,w2,w3,w4,w5,w6)
    evalscore = []
    for i in range(len(x)):
        board_state[x[i],y[i]]=1

        ft1, ft2, ft3, ft4, ft5, ft6 = evalboard(board_state)
        feature_list.append([ft1,ft2,ft3,ft4,ft5,ft6])
        
        evalscore.append(w0 + w1*ft1 + w2*ft2 + w3*ft3 + w4*ft4 + w5*ft5 + w6*ft6)
        feature_list[i].extend([w0 + w1*ft1 + w2*ft2 + w3*ft3 + w4*ft4 + w5*ft5 + w6*ft6])
        #print("features",feature_list)
        #print("evalscore",evalscore)
        board_state = np.copy(old_board_state)
        

    position_index = np.argmax(np.asarray(evalscore))
    optimal_feature = feature_list[position_index]
    #print("Optimal feature is ", optimal_feature)
    #print("position_index is ",position_index)

    machine1_position = x[position_index]*3 + y[position_index]
    #moves.append(machine1_position)
    
    return(machine1_position, max(evalscore), optimal_feature)




### the below code is for the machine which doesnt learn - makes random moves 

In [5]:

def machine2_move(game_moves):
    while(True):
        machine2_position = randint(0,8)
        if machine2_position not in game_moves:
            #game_board[machine2_position // 3, machine2_position % 3] = 2
            #moves.append(machine2_position)
            
            #clear_output()
            print("Machine 2's move")
            break
    return(machine2_position)



### Below code is for updating weights after each training game is played

In [6]:
def weight_update(X, y, game_counter,w0,w1,w2,w3,w4,w5,w6):
    print("Game counter", game_counter)
    f1 = [sublist[0] for sublist in X]
    f2 = [sublist[1] for sublist in X]
    f3 = [sublist[2] for sublist in X]
    f4 = [sublist[3] for sublist in X]
    f5 = [sublist[4] for sublist in X]
    f6 = [sublist[5] for sublist in X]
    Vestimate = [sublist[6] for sublist in X]
    Vtrain_flattened = [item for sublist in y for item in sublist]
    #print("Vtrain_flattened is",Vtrain_flattened)
    
    lr = 0.1
    print("length of f1",len(f1))
    w0_old = w0
    w1_old = w1
    w2_old = w2
    w3_old = w3
    w4_old = w4
    w5_old = w5
    w6_old = w6
    
    for i in range(0, len(f1)):
        w0_new = w0_old + lr*(Vtrain_flattened[i] - Vestimate[i])
        w1_new = w1_old + lr*(Vtrain_flattened[i] - Vestimate[i])*f1[i]
        w2_new = w2_old + lr*(Vtrain_flattened[i] - Vestimate[i])*f2[i]
        w3_new = w3_old + lr*(Vtrain_flattened[i] - Vestimate[i])*f3[i]
        w4_new = w4_old + lr*(Vtrain_flattened[i] - Vestimate[i])*f4[i]
        w5_new = w5_old + lr*(Vtrain_flattened[i] - Vestimate[i])*f5[i]
        w6_new = w6_old + lr*(Vtrain_flattened[i] - Vestimate[i])*f6[i]
        #print("Vtrain_flattened",Vtrain_flattened[i])
        w0_old = w0_new
        w1_old = w1_new
        w2_old = w2_new
        w3_old = w3_new
        w4_old = w4_new
        w5_old = w5_new
        w6_old = w6_new
       
        
        if i==(len(f1)-1):
            break

        Vestimate[i+1] = w0_new + w1_new*f1[i+1] + w2_new*f2[i+1] + w3_new*f3[i+1] + w4_new*f4[i+1] + w5_new*f5[i+1] + w6_new*f6[i+1]
    #print(w0_new,w1_new,w2_new,w3_new,w4_new,w5_new,w6_new)
    return(w0_new,w1_new,w2_new,w3_new,w4_new,w5_new,w6_new)




#### For printing the game board

In [7]:
def print_game_board(game_board):
    print_board = np.copy(game_board)
    print_board_list = print_board.ravel().tolist()
    replace_dict = {1:'X',2:'O',0:" "}
    print_board_XO = [replace_dict.get(n) for n in print_board_list]
    
    print("     |     |     ")
    print("  "+print_board_XO[0]+"  |  "+print_board_XO[1]+"  |  "+print_board_XO[2]+"  ")
    print("_____|_____|_____")
    print("     |     |     ")
    print("  "+print_board_XO[3]+"  |  "+print_board_XO[4]+"  |  "+print_board_XO[5]+"  ")
    print("_____|_____|_____")
    print("     |     |     ")
    print("  "+print_board_XO[6]+"  |  "+print_board_XO[7]+"  |  "+print_board_XO[8]+"  ")
    print("     |     |     ")

### Below code is for the whole training process

In [8]:
##List of winning moves - whenever any combination in this achieved it is game over
winning_moves = [[0,1,2],[0,3,6],[0,4,8],[1,4,7],[2,5,8],[2,4,6],[3,4,5],[6,7,8]]
positions = []
start_index = 0
game_feature = []

Vtrain = []

## Initializing weights for the learning. This can be made random also instead of a fixed 0.5
w0 = w1 = w2 = w3 = w4 = w5 = w6 = 0.5
print("intial wights",w0,w1,w2,w3,w4,w5,w6)


for m in range(0,1000):
    
    ##Initializing different lists - moves contain overall moves made by both the agents
    ## This is to prevent repetition of moves
    ## The individual agent moves are to compare with the winning moves list
    ## The board is initialized with zeros
    
    moves = []
    machine1_moves = []
    machine2_moves = []
    Vest = []
    feature_list = []
    game_feature = []
    Vtrain=[]
    game_board = np.zeros([3,3])
    print(game_board)
    
    ## This loop is for the game play - a maximum of 9 moves can be made in a game
    for i in range(0,9):
        #game_feature = []
        ## Initializing a game over flag to 0
        GO = 0
        
        ## Calling the function machine1_move for making the moves of the agent 1
        ## and receiving the move position in machine1_position variable
        ## Passing the weights as function arguments 
        machine1_position, vestimate, feature_list = machine1_move(game_board, w0, w1, w2, w3, w4, w5, w6)
        print("feature_list is", feature_list)
        print("Vestimate is ", vestimate)
        Vest.append(vestimate)
        
        ##Updating the game board with the machine1 move position
        game_board[machine1_position // 3, machine1_position % 3] = 1
        print("Machine 1's move")
        print(game_board)
        
        ## Creating a copy of game board into mc1
        ## Pushing the game board status into the positions list
        ## this will useful later when creating the dataframe of game board positions
        
        #mc1 = np.copy(game_board)
        #positions.append(mc1.ravel().tolist())
        game_feature.append(feature_list)
 
        
        ## Pushing the moves machine 1 into the moves list and the machine1_moves list
        moves.append(machine1_position)
        print("Moves array",moves)
        machine1_moves.append(machine1_position)
        machine1_counter = 0
        
        ## If more than two moves are made, then keep checking in the winning moves list
        if len(machine1_moves) > 2:
            for j in range(len(winning_moves)):
                #human_counter = 0
                ## For each set (3 positions) of winning moves, the values in the machine1_moves
                ## are checked with the winning moves set - and if there is a match of 3, the game is 
                ## declared won by machine 1
                machine1_counter = [a for a in machine1_moves if a in winning_moves[j]]
                if len(machine1_counter)==3:
                    print("Game Over")
                    print("Machine1 won")
                    GO = 1
                    break
        
        ## If game over, the GO flag will be set to 1 in the previous code block and the previous loop is exited.
        ## Now, for all the gameboard arrays in the position list (which includes moves by machine 2 as well)
        ## a 1 is added - to indicate that the game was won by agent 1.
        ## Else if the number of moves is 9 without the GO flag set to 1, then it is a tie game and hence '0' is 
        ## added to the position arrays
        if GO == 1:
            Vest.append(100)
            Vest.pop(0)
            Vtrain.append(Vest)

            break
            
        elif len(machine1_moves)+len(machine2_moves) == 9:
            print("Game Tie")
            Vest.append(0)
            Vest.pop(0)
            Vtrain.append(Vest)
            break
            
            
        ## Calling the function machine2_move for making the moves of the agent 2
        ## and receiving the move position in machine2_position variable
        
        
        machine2_position = machine2_move(moves)
        
        game_board[machine2_position // 3, machine2_position % 3] = 2
        print(game_board)
        
        mc2 = np.copy(game_board)
        #positions.append(mc2.ravel().tolist())
        
        moves.append(machine2_position)
        machine2_moves.append(machine2_position)
        machine2_counter = 0
        
        if len(machine2_moves) > 2:
            for h in range(len(winning_moves)):
                #machine_counter = 0
                machine2_counter = [b for b in machine2_moves if b in winning_moves[h]]
                if len(machine2_counter)==3:
                    print("Game Over")
                    print("Machine2 won")
                    GO = 2
                    break

        if GO == 2:
            Vest.append(-100)
            Vest.pop(0)
            Vtrain.append(Vest)
            break
    #print("GAME FEATURE",game_feature)
    w0,w1,w2,w3,w4,w5,w6 = weight_update(game_feature, Vtrain,m,w0,w1,w2,w3,w4,w5,w6)
    print("weights after update",w0,w1,w2,w3,w4,w5,w6)

intial wights 0.5 0.5 0.5 0.5 0.5 0.5 0.5
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
feature_list is [0, 0, 4, 0, 0, 0, 2.5]
Vestimate is  2.5
Machine 1's move
[[0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]
Moves array [4]
Machine 2's move
[[0. 0. 0.]
 [2. 1. 0.]
 [0. 0. 0.]]
feature_list is [1, 0, 4, 1, 0, 0, 3.5]
Vestimate is  3.5
Machine 1's move
[[0. 0. 1.]
 [2. 1. 0.]
 [0. 0. 0.]]
Moves array [4, 3, 2]
Machine 2's move
[[0. 0. 1.]
 [2. 1. 0.]
 [0. 2. 0.]]
feature_list is [2, 0, 2, 2, 0, 0, 3.5]
Vestimate is  3.5
Machine 1's move
[[0. 1. 1.]
 [2. 1. 0.]
 [0. 2. 0.]]
Moves array [4, 3, 2, 7, 1]
Machine 2's move
[[0. 1. 1.]
 [2. 1. 0.]
 [0. 2. 2.]]
feature_list is [2, 1, 0, 1, 0, 0, 2.5]
Vestimate is  2.5
Machine 1's move
[[0. 1. 1.]
 [2. 1. 1.]
 [0. 2. 2.]]
Moves array [4, 3, 2, 7, 1, 8, 5]
Machine 2's move
[[0. 1. 1.]
 [2. 1. 1.]
 [2. 2. 2.]]
Game Over
Machine2 won
Game counter 0
length of f1 4
weights after update -9.7527 -20.035400000000003 -9.713700000000001 0.282 -9.821700000000002 0.5 0.5
[[0

Moves array [4]
Machine 2's move
[[0. 2. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]
feature_list is [0, 0, 4, 1, 0, 0, 23.447896945459938]
Vestimate is  23.447896945459938
Machine 1's move
[[0. 2. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]
Moves array [4, 1, 7]
Machine 2's move
[[0. 2. 0.]
 [0. 1. 0.]
 [0. 1. 2.]]
feature_list is [0, 0, 3, 1, 0, 0, 20.332074425001643]
Vestimate is  20.332074425001643
Machine 1's move
[[1. 2. 0.]
 [0. 1. 0.]
 [0. 1. 2.]]
Moves array [4, 1, 7, 8, 0]
Machine 2's move
[[1. 2. 0.]
 [2. 1. 0.]
 [0. 1. 2.]]
feature_list is [0, 0, 1, 0, 0, 0, 20.263825731689522]
Vestimate is  20.263825731689522
Machine 1's move
[[1. 2. 0.]
 [2. 1. 1.]
 [0. 1. 2.]]
Moves array [4, 1, 7, 8, 0, 3, 5]
Machine 2's move
[[1. 2. 0.]
 [2. 1. 1.]
 [2. 1. 2.]]
feature_list is [0, 0, 0, 0, 0, 0, 17.148003211231227]
Vestimate is  17.148003211231227
Machine 1's move
[[1. 2. 1.]
 [2. 1. 1.]
 [2. 1. 2.]]
Moves array [4, 1, 7, 8, 0, 3, 5, 6, 2]
Game Tie
Game counter 17
length of f1 5
weights after update 15.079135312911

Machine1 won
Game counter 38
length of f1 3
weights after update 36.02007128108454 19.81221286999581 -14.470569799529972 9.80877852875625 -4.156909251340194 32.33684016282785 0.5
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
feature_list is [0, 0, 4, 0, 0, 0, 75.25518539610954]
Vestimate is  75.25518539610954
Machine 1's move
[[0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]
Moves array [4]
Machine 2's move
[[0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 2.]]
feature_list is [1, 0, 3, 1, 0, 0, 81.10171048600891]
Vestimate is  81.10171048600891
Machine 1's move
[[0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 2.]]
Moves array [4, 8, 2]
Machine 2's move
[[0. 2. 1.]
 [0. 1. 0.]
 [0. 0. 2.]]
feature_list is [0, 0, 2, 0, 1, 0, 87.9744685014249]
Vestimate is  87.9744685014249
Machine 1's move
[[0. 2. 1.]
 [0. 1. 0.]
 [1. 0. 2.]]
Moves array [4, 8, 2, 1, 6]
Game Over
Machine1 won
Game counter 39
length of f1 3
weights after update 37.25925794379745 19.73944040985049 -14.470569799529972 13.38368441201662 -4.229681711485514 33.06414677669614 0.5
[[

weights after update 47.415862051631365 3.3884058808483726 -33.75911623111807 -6.796868984487789 0.34544210568947875 35.38749626853316 0.5
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
feature_list is [0, 0, 2, 0, 0, 0, 33.82212408265579]
Vestimate is  33.82212408265579
Machine 1's move
[[0. 1. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Moves array [1]
Machine 2's move
[[0. 1. 0.]
 [0. 0. 0.]
 [0. 0. 2.]]
feature_list is [1, 0, 1, 2, 0, 0, 44.69828315937091]
Vestimate is  44.69828315937091
Machine 1's move
[[0. 1. 0.]
 [0. 0. 0.]
 [0. 1. 2.]]
Moves array [1, 8, 7]
Machine 2's move
[[0. 1. 0.]
 [0. 0. 0.]
 [2. 1. 2.]]
feature_list is [2, 0, 0, 2, 0, 0, 54.88355802470707]
Vestimate is  54.88355802470707
Machine 1's move
[[1. 1. 0.]
 [0. 0. 0.]
 [2. 1. 2.]]
Moves array [1, 8, 7, 6, 0]
Machine 2's move
[[1. 1. 0.]
 [2. 0. 0.]
 [2. 1. 2.]]
feature_list is [1, 0, 0, 1, 1, 0, 86.53720630670239]
Vestimate is  86.53720630670239
Machine 1's move
[[1. 1. 1.]
 [2. 0. 0.]
 [2. 1. 2.]]
Moves array [1, 8, 7, 6, 0, 3, 2]
Ga

Moves array [4, 3, 2, 7, 6]
Game Over
Machine1 won
Game counter 76
length of f1 3
weights after update 64.13497067473958 11.169236717517801 -29.49976362153226 4.594583054106057 -3.7460038781459115 21.377969432942518 0.5
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
feature_list is [0, 0, 4, 0, 0, 0, 82.5133028911638]
Vestimate is  82.5133028911638
Machine 1's move
[[0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]
Moves array [4]
Machine 2's move
[[0. 0. 0.]
 [2. 1. 0.]
 [0. 0. 0.]]
feature_list is [1, 0, 4, 1, 0, 0, 89.9365357305357]
Vestimate is  89.9365357305357
Machine 1's move
[[0. 0. 1.]
 [2. 1. 0.]
 [0. 0. 0.]]
Moves array [4, 3, 2]
Machine 2's move
[[0. 0. 1.]
 [2. 1. 0.]
 [0. 0. 2.]]
feature_list is [0, 0, 2, 0, 1, 0, 94.7021062158942]
Vestimate is  94.7021062158942
Machine 1's move
[[0. 0. 1.]
 [2. 1. 0.]
 [1. 0. 2.]]
Moves array [4, 3, 2, 8, 6]
Game Over
Machine1 won
Game counter 77
length of f1 3
weights after update 64.66045312812815 10.38384418336043 -29.49976362153226 5.559409460442811 -4.5313

...

feature_list is [1, 0, 0, 0, 1, 0, 97.83269440939492]
Vestimate is  97.83269440939492
Machine 1's move
[[1. 0. 1.]
 [0. 1. 2.]
 [2. 2. 1.]]
Moves array [4, 7, 0, 5, 2, 6, 8]
Game Over
Machine1 won
Game counter 896
length of f1 4
weights after update 70.16055666493305 9.04566448405071 -22.354786010879003 5.230359973764464 -4.438971738837068 16.16910252210079 0.5
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
feature_list is [0, 0, 4, 0, 0, 0, 91.08199655999091]
Vestimate is  91.08199655999091
Machine 1's move
[[0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]
Moves array [4]
Machine 2's move
[[0. 0. 2.]
 [0. 1. 0.]
 [0. 0. 0.]]
feature_list is [1, 0, 3, 1, 0, 0, 90.45832933144008]
Vestimate is  90.45832933144008
Machine 1's move
[[1. 0. 2.]
 [0. 1. 0.]
 [0. 0. 0.]]
Moves array [4, 2, 0]
Machine 2's move
[[1. 0. 2.]
 [0. 1. 0.]
 [0. 2. 0.]]
feature_list is [0, 0, 2, 0, 1, 0, 96.79037913456277]
Vestimate is  96.79037913456277
Machine 1's move
[[1. 0. 2.]
 [0. 1. 0.]
 [0. 2. 1.]]
Moves array [4, 2, 0, 7, 8]
Game 

weights after update 70.0924639052344 10.596803158903251 -22.354786010879003 3.536959460957167 -4.335012186090932 18.477925765305663 0.5
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
feature_list is [0, 0, 4, 0, 0, 0, 84.24030174906306]
Vestimate is  84.24030174906306
Machine 1's move
[[0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]
Moves array [4]
Machine 2's move
[[0. 0. 0.]
 [0. 1. 0.]
 [2. 0. 0.]]
feature_list is [1, 0, 3, 1, 0, 0, 86.96513326091822]
Vestimate is  86.96513326091822
Machine 1's move
[[1. 0. 0.]
 [0. 1. 0.]
 [2. 0. 0.]]
Moves array [4, 6, 0]
Machine 2's move
[[1. 2. 0.]
 [0. 1. 0.]
 [2. 0. 0.]]
feature_list is [0, 0, 2, 0, 1, 0, 95.6443085924544]
Vestimate is  95.6443085924544
Machine 1's move
[[1. 2. 0.]
 [0. 1. 0.]
 [2. 0. 1.]]
Moves array [4, 6, 0, 1, 8]
Game Over
Machine1 won
Game counter 918
length of f1 3
weights after update 70.70938819209124 11.1104925955157 -22.354786010879003 5.829463773654338 -3.821322749478484 18.308677464364546 0.5
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
featur

Vestimate is  83.0603720937659
Machine 1's move
[[0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]
Moves array [4]
Machine 2's move
[[0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 2.]]
feature_list is [1, 0, 3, 1, 0, 0, 85.62708825271669]
Vestimate is  85.62708825271669
Machine 1's move
[[0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 2.]]
Moves array [4, 8, 2]
Machine 2's move
[[0. 0. 1.]
 [0. 1. 2.]
 [0. 0. 2.]]
feature_list is [0, 0, 3, 0, 1, 0, 99.80572793076108]
Vestimate is  99.80572793076108
Machine 1's move
[[0. 0. 1.]
 [0. 1. 2.]
 [1. 0. 2.]]
Moves array [4, 8, 2, 5, 6]
Game Over
Machine1 won
Game counter 938
length of f1 3
weights after update 61.44382011807004 14.46760851745995 -22.354786010879003 5.473693206823988 -4.342766199764686 20.73666350057744 0.5
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
feature_list is [0, 0, 4, 0, 0, 0, 83.338592945366]
Vestimate is  83.338592945366
Machine 1's move
[[0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]
Moves array [4]
Machine 2's move
[[0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 2.]]
feature_list is [1, 0, 3, 1

Vestimate is  87.84738134993069
Machine 1's move
[[0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]
Moves array [4]
Machine 2's move
[[0. 0. 0.]
 [2. 1. 0.]
 [0. 0. 0.]]
feature_list is [1, 0, 4, 1, 0, 0, 92.65779552889181]
Vestimate is  92.65779552889181
Machine 1's move
[[0. 0. 1.]
 [2. 1. 0.]
 [0. 0. 0.]]
Moves array [4, 3, 2]
Machine 2's move
[[0. 0. 1.]
 [2. 1. 0.]
 [2. 0. 0.]]
feature_list is [2, 0, 2, 1, 0, 0, 90.37526808964353]
Vestimate is  90.37526808964353
Machine 1's move
[[1. 0. 1.]
 [2. 1. 0.]
 [2. 0. 0.]]
Moves array [4, 3, 2, 6, 0]
Machine 2's move
[[1. 0. 1.]
 [2. 1. 2.]
 [2. 0. 0.]]
feature_list is [1, 0, 1, 0, 1, 0, 97.28645972706381]
Vestimate is  97.28645972706381
Machine 1's move
[[1. 0. 1.]
 [2. 1. 2.]
 [2. 0. 1.]]
Moves array [4, 3, 2, 6, 0, 5, 8]
Game Over
Machine1 won
Game counter 959
length of f1 4
weights after update 68.8159637158168 9.540805099159565 -22.354786010879003 5.639031760686896 -2.3804658358069086 16.59374517716994 0.5
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
feat

weights after update 61.04360692396791 10.515522038628891 -22.354786010879003 5.252004412230713 -3.696146507102326 16.437698740165597 0.5
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
feature_list is [0, 0, 4, 0, 0, 0, 82.05162457289076]
Vestimate is  82.05162457289076
Machine 1's move
[[0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]
Moves array [4]
Machine 2's move
[[0. 0. 0.]
 [2. 1. 0.]
 [0. 0. 0.]]
feature_list is [1, 0, 4, 1, 0, 0, 88.87100010441733]
Vestimate is  88.87100010441733
Machine 1's move
[[0. 0. 1.]
 [2. 1. 0.]
 [0. 0. 0.]]
Moves array [4, 3, 2]
Machine 2's move
[[0. 2. 1.]
 [2. 1. 0.]
 [0. 0. 0.]]
feature_list is [3, 0, 1, 1, 0, 0, 94.14603094498297]
Vestimate is  94.14603094498297
Machine 1's move
[[0. 2. 1.]
 [2. 1. 0.]
 [0. 0. 1.]]
Moves array [4, 3, 2, 1, 8]
Machine 2's move
[[0. 2. 1.]
 [2. 1. 0.]
 [2. 0. 1.]]
feature_list is [1, 0, 0, 0, 1, 0, 87.9968277027624]
Vestimate is  87.9968277027624
Machine 1's move
[[1. 2. 1.]
 [2. 1. 0.]
 [2. 0. 1.]]
Moves array [4, 3, 2, 1, 8, 6, 0]
Game 

## Execute below two blocks for playing / testing with human player

##### This is for making the moves of the human player

In [9]:
def human_move(moves_entered):
    move_entered = 0
    while (move_entered == 0):
        human_position = int(input("enter your move 1 - 9 ")) - 1
        if human_position > 8 or human_position < 0:
            print("Enter your move between 1 - 9")
        elif human_position not in moves_entered:
            #game_board[position // 3, position % 3] = 1
            move_entered = 2
            #clear_output()
            #print(game_board)
        else:
            print("Enter into a blank cell")
    return(human_position)

#### Pitting the human with the smart agent

In [24]:

#positions = []
winning_moves = [[0,1,2],[0,3,6],[0,4,8],[1,4,7],[2,5,8],[2,4,6],[3,4,5],[6,7,8]]
#start_index = 0


real_game_moves = []
machine_moves = []
human_moves = []
real_game_board = np.zeros([3,3])
#print(real_game_board)
print_game_board(real_game_board)


for i in range(0,9):
    GO = 0
    
    gb_copy = np.copy(real_game_board)
    
    machine_position, vestimate, feature_list = machine1_move(real_game_board, w0, w1, w2, w3, w4, w5, w6)
       
    #machine_position = machine_move(gb_copy.ravel())
    #print(machine_position)
    real_game_board[machine_position // 3, machine_position % 3] = 1
    clear_output()
    #print(real_game_board)
    print_game_board(real_game_board)
    real_game_moves.append(machine_position)
    machine_moves.append(machine_position)
    
    if len(machine_moves) > 2:
        for j in range(len(winning_moves)):
            
            machine_counter = [a for a in machine_moves if a in winning_moves[j]]
            if len(machine_counter)==3:
                print("Game Over")
                print("Machine won")
                GO = 1
                break
    
    if GO == 1:
        break
            
    elif len(machine_moves)+len(human_moves) == 9:
        print("Game Tie")
        break
      
    
    human_game_position = human_move(real_game_moves)
    real_game_board[human_game_position // 3, human_game_position % 3] = 2
    clear_output()
    #print(real_game_board)
    print_game_board(real_game_board)
    real_game_moves.append(human_game_position)
    human_moves.append(human_game_position)
    
    if len(human_moves) > 2:
        for h in range(len(winning_moves)):
            #machine_counter = 0
            human_counter = [b for b in human_moves if b in winning_moves[h]]
            if len(human_counter)==3:
                print("Game Over")
                print("Human won")
                GO = 1
                break
    
    if GO == 1:
        break   
    
    

     |     |     
  X  |     |  X  
_____|_____|_____
     |     |     
  O  |  X  |     
_____|_____|_____
     |     |     
  X  |  O  |  O  
     |     |     
Game Over
Machine won
